In [25]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder

Importar y Cargar

In [34]:
# Construimos la ruta dinámica al archivo
file_path = os.path.join("..", "data", "citas_medicas.csv")

# Leemos el CSV
df = pd.read_csv(file_path)

# Verificamos que cargó mostrando las primeras 5 filas
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [35]:
# Renombrar cabeceras (Estandarización)
df.rename(columns={
    'AppointmentID': 'appointment_id',
    'PatientId': 'patient_id',
    'Gender': 'gender',
    'ScheduledDay': 'scheduled_day',
    'AppointmentDay': 'appointment_day',
    'Age': 'age',
    'Neighbourhood': 'neighbourhood',
    'Scholarship': 'scholarship',
    'Hipertension': 'hypertension',
    'Diabetes': 'diabetes',
    'Alcoholism': 'alcoholism',
    'Handcap': 'handicap',
    'SMS_received': 'sms_received',
    'No-show': 'no_show'
}, inplace=True)

# Arreglar el Target
# Convertimos "Yes/No" a 1/0 para poder hacer matemáticas con ello
df['target'] = df['no_show'].apply(lambda x: 1 if x == 'Yes' else 0)
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,no_show,target
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No,0
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No,0
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,0
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No,0


In [36]:
# Convertimos las columnas de fecha (que eran texto) a objetos de tiempo real
df['scheduled_day'] = pd.to_datetime(df['scheduled_day']).dt.normalize()
df['appointment_day'] = pd.to_datetime(df['appointment_day']).dt.normalize()

# ORDENAR: Primero por paciente, luego por fecha de cita.
df = df.sort_values(by=['patient_id', 'appointment_day'])

Feature engineering

In [37]:
# 1. Historial de Faltas (Numerador)
# Agrupamos por paciente -> Sumamos faltas acumuladas -> Desplazamos 1 fila
df['previous_misses'] = df.groupby('patient_id')['target'].transform(lambda x: x.cumsum().shift(1)).fillna(0)

# 2. Historial de Citas Totales (Denominador)
# cumcount cuenta: Cita 0, Cita 1, Cita 2...
df['previous_appts'] = df.groupby('patient_id').cumcount()

# 3. Ratio Histórico (Feature Final)
# Faltas / Total. Si es la primera cita (0/0), ponemos 0.
df['history_ratio'] = (df['previous_misses'] / df['previous_appts']).fillna(0)

print("Historial calculado.")
# Validamos visualmente
df[['patient_id', 'appointment_day', 'target', 'history_ratio']].head()

Historial calculado.


,patient_id,appointment_day,target,history_ratio
100517,39217.84439,2016-06-03 00:00:00+00:00,0,0.0
105430,43741.75652,2016-06-01 00:00:00+00:00,0,0.0
3950,93779.52927,2016-05-18 00:00:00+00:00,0,0.0
73303,141724.16655,2016-05-02 00:00:00+00:00,0,0.0
73228,537615.28476,2016-05-06 00:00:00+00:00,0,0.0


Encoding

In [38]:
# 1. Feature: Días de anticipación (Reserva vs Cita)
df['days_wait'] = (df['appointment_day'] - df['scheduled_day']).dt.days

# 2. Feature: Día de la Semana (Lunes=0 ... Domingo=6)
df['day_of_week'] = df['appointment_day'].dt.dayofweek

# 3. Encoding de Texto a Números
# Género: F->0, M->1
df['gender_enc'] = df['gender'].map({'F': 0, 'M': 1})

# Barrio: Texto -> ID Numérico
le = LabelEncoder()
df['neighbourhood_enc'] = le.fit_transform(df['neighbourhood'])

# Limpieza Lógica (Data Quality Check)
# Eliminamos registros con errores de sistema (edad negativa o fechas invertidas)
df_clean = df[(df['age'] >= 0) & (df['days_wait'] >= 0)].copy()

print(f"Variables generadas. Registros limpios: {df_clean.shape[0]}")

Variables generadas. Registros limpios: 110521


In [39]:
# Definimos las columnas FINALES que queremos en nuestro modelo
# Descartamos los IDs y los textos originales
final_columns = [
    'gender_enc', 
    'age', 
    'neighbourhood_enc', 
    'scholarship', 
    'hypertension', 
    'diabetes', 
    'alcoholism', 
    'handicap', 
    'sms_received', 
    'days_wait', 
    'day_of_week', 
    'history_ratio',
    'target' # Lo que queremos predecir
]

# Creamos el DataFrame final (Silver Layer)
df_silver = df_clean[final_columns]

# Guardamos
output_path = os.path.join("..", "data", "processed_citas_v2.csv")
df_silver.to_csv(output_path, index=False)

print(f"Archivo procesado guardado en: {output_path}")
print("Columnas finales:", df_silver.columns.tolist())

Archivo procesado guardado en: ..\data\processed_citas_v2.csv
Columnas finales: ['gender_enc', 'age', 'neighbourhood_enc', 'scholarship', 'hypertension', 'diabetes', 'alcoholism', 'handicap', 'sms_received', 'days_wait', 'day_of_week', 'history_ratio', 'target']
